In [22]:
import ipywidgets as widgets
import anywidget
import traitlets
import jupyter
from tweet_browser_test import tweet_browser as tb
import voila
from matplotlib import pyplot as plt
from IPython.display import display
import pandas as pd
import io

TWEETS_PER_PAGE = 20

out = widgets.Output()

def startSession(file):
    if file['type'] == 'xls':
        df = pd.read_excel(io.BytesIO(file.content))
    else:
        df = pd.read_csv(io.BytesIO(file.content))
    db = tb.DataBaseSim(df)
    s = tb.Session(db)
    browser = Browser(s, out)
    

def selectColumns (row, colHeaders: list):
    result = []
    for j in colHeaders:
        result.append(row[s.headerDict[j]])
    return result

class SearchBar(anywidget.AnyWidget):
    _esm = "anywidget/searchBar.js"
    _css = "anywidget/searchBar.css"
    value = traitlets.List([]).tag(sync=True)
    totalItems = traitlets.Int(0).tag(sync=True)
    
class TweetDisplay(anywidget.AnyWidget):
    _esm = "anywidget/tweetDisplay.js"
    _css = "anywidget/tweetDisplay.css"
    value = traitlets.List().tag(sync=True)
    
class DatasetDisplay(anywidget.AnyWidget):
    _esm = "anywidget/datasetDisplayPart.js"
    _css = "anywidget/datasetDisplay.css"
    size = traitlets.Int().tag(sync=True)
    fileName = traitlets.Unicode().tag(sync=True)
    
class PageSelect(anywidget.AnyWidget):
    _esm = "anywidget/pageSelect.js"
    _css = "anywidget/pageSelect.css"
    tweetsPerPage = traitlets.Int(TWEETS_PER_PAGE).tag(sync=True)
    value = traitlets.CInt(1).tag(sync=True)
    totalTweets = traitlets.Int().tag(sync=True)
    changeSignal = traitlets.Int(0).tag(sync=True)
    
# File upload
fileUp = widgets.widgets.FileUpload(
    accept='.csv, .txt, .xls, .tsv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description='Change Dataset'
)
fileUp.add_class("change-input")
datasetDisplayCustom = DatasetDisplay()
datasetDisplayCustom.add_class("text-block")
fileUpBar = widgets.Box([datasetDisplayCustom, fileUp])
fileUpBar.add_class("dataset-display")

class Browser:
    def __init__(self, s, out):
        self.s = s
        self.out = out
        self.operators = [None] * 5
        self.colHeaders = list(s.headerDict.keys())
        self.createWidgets()
        self.resetDisplay()
        
    def resetDisplay(self, b = None):
        self.out.clear_output(True)
        with out:
            display(self.searchBar)
            display(self.datasetDisplay)
            display(self.pageSelect)
            display(self.searchButton)
            display(self.tweetDisplay)
            
    def searchKeyword(self, b):
        if(self.searchBar.totalItems == 0):
            return
        if(self.s.currentSet.size < self.s.length):
            self.s.back()
        with out:
            print(self.searchBar.value)
        self.s.searchKeyword(self.searchBar.value, False)
        self.getTweets(b)
    
    
    def getTweets(self, b, pageSize = TWEETS_PER_PAGE):
        #self.resetDisplay()
        dataSet = self.s.getCurrentSubset()
        tempArr = []
        
        pageNum = self.pageSelect.value
        
        self.pageSelect.totalTweets = len(dataSet)
        self.pageSelect.changeSignal += 1
        
        for i in range((pageNum-1) * pageSize, min(len(dataSet), pageNum * pageSize)):
            tempArr.append(self.s.getCurrentSubset().iloc[i].to_json())
        
        self.tweetDisplay.value = tempArr

        
    def createWidgets(self):
        self.searchBar = SearchBar() 
        self.searchButton = widgets.Button(description='Search')
        self.searchButton.on_click(self.searchKeyword)
        self.tweetDisplay = TweetDisplay()
        self.datasetDisplay = fileUpBar
        self.pageSelect = PageSelect()
        self.pageSelect.observe(self.changePage, names=["value"])
    
    def changePage(self, change):
        self.getTweets(None)
        
    
def fileHandler(change):
    startSession(fileUp.value[0])

        
with out:
    display(fileUpBar)
    #display(DatasetDisplay())


fileUp.observe(fileHandler, names=["value"])

out

Output()